# GPU yeast UTR training with Pytorch

This is a Pytorch version. (Python 3)

In [1]:
import torch
print('GPU available to Torch: {}'.format(torch.cuda.is_available()))

GPU available to Torch: True


In [13]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
#import torch

In [8]:
!pip install tqdm ipywidgets

In [2]:
import sys
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import random


In [4]:
import ipywidgets
from ipywidgets import IntProgress
from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm

Define a class that handles DNA datasets. Crucially, it includes a function __getitem__ which is used in the DataLoader later.

We also define a class for the neural network model.

In [5]:
class DNADataset(Dataset):

    def __init__(self, df, seq_len):
        self.data = df
        self.bases = ['A','C','G','T']
        self.base_dict = dict(zip(self.bases,range(4))) # {'A' : 0, 'C' : 1, 'G' : 2, 'T' : 3}
        self.total_width = seq_len + 20

    def __len__(self):
        return (self.data.shape[0])

    def __getitem__(self, idx):
        seq = self.data.iloc[idx].UTR
        X = np.zeros((1, 4, self.total_width))
        y = self.data.iloc[idx].growth_rate
        for b in range(len(seq)):
            # this will assign a 1 to the appropriate base and position for this UTR sequence
            X[0, self.base_dict[seq[b]], int(b + round((self.total_width - len(seq))/2.))] = 1.
        return(seq, X, y)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=128, kernel_size=(4, 13))
        self.dropout = nn.Dropout(p=0.15)
        self.conv2 = nn.Conv2d(128, 128, (1,13))
        self.fc1 = nn.Linear(128 * 1 * 34, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = x.view(-1, 128 * 1 * 34)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()

net = net.cuda() # to run it on GPU, if available

# Loss function etc.
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())

Look at the configuration of the net.

In [6]:
net

Net(
  (conv1): Conv2d(1, 128, kernel_size=(4, 13), stride=(1, 1))
  (dropout): Dropout(p=0.15)
  (conv2): Conv2d(128, 128, kernel_size=(1, 13), stride=(1, 1))
  (fc1): Linear(in_features=4352, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

Read the data and sort by t0 as in the original Keras notebook. I hope that the validation set will also come out the same way as in the Keras notebook. 

In [7]:
#df = pd.read_csv("drive/Mikael-Colab/Random_UTRs.csv")
import pandas as pd
df = pd.read_csv("https://github.com/animesh/2017---Deep-learning-yeast-UTRs/raw/master/Data/Random_UTRs.csv.gz")
sorted_inds = df.sort_values('t0').index.values
train_inds = sorted_inds[:int(0.95*len(sorted_inds))] # 95% of the data as the training set
# Separate out test set
test_inds = sorted_inds[int(0.95*len(sorted_inds)):] # UTRs with most reads at time point 0 as the test set
# Get validation set
val_idx = int(0.9*len(train_inds))
val_inds = train_inds[val_idx:]
train_inds = train_inds[:val_idx]


In [8]:
#df = pd.read_csv("https://github.com/animesh/2017---Deep-learning-yeast-UTRs/raw/master/Data/Random_UTRs.csv.gz")
df.head()

,Unnamed: 0,UTR,growth_rate,t0,t1
0,0,AAAAAAAAAACATAATAACGATGATCAGTTAAAATCATAGTCTAAG...,-1.237065,14,3
1,1,AAAAAAAAAAGACTACAACAGATTGTAGTGGCGGACCAGTGTGCCT...,1.288663,14,49
2,2,AAAAAAAAAATATGGGGCCCTGTTCCAAAGATACCTCAATTTCATA...,-0.608457,13,6
3,3,AAAAAAAAAATCTCTGGCCCGATTATACTGGAGCTAATGTAAAATT...,-1.093964,12,3
4,4,AAAAAAAAACATAAATATGAAGGCCTGACATTATAAATAACTTACC...,-0.048841,7,6


Now Pytorch's DataLoader functionality is used to extract batches of the data.

In [9]:
train_data = DNADataset(df.iloc[train_inds], seq_len=50)
val_data = DNADataset(df.iloc[val_inds], seq_len=50)
test_data = DNADataset(df.iloc[test_inds], seq_len=50)

train_data_loader = DataLoader(train_data, batch_size=32,
                        shuffle=True, num_workers=4)

val_data_loader = DataLoader(val_data, batch_size=32) # Validate everything in one batch?!
#seq_val, X_val, y_val = next(iter(val_data_loader))

test_data_loader = DataLoader(test_data, batch_size=32) # Validate everything in one batch?!
#seq_test, X_test, y_test = next(iter(test_data_loader))


The training loop contains an evaluation every 1000 steps. I did not bother to make an early stopping functionality although that would have been nice.

In [10]:
for epoch in range(10):
    #sys.stderr.write(('EPOCH ' + str(epoch)))
    #sys.stderr.flush()
    for i_batch, sampled_batch in enumerate(tqdm(train_data_loader)):

        sequence, transformed_sequence, growth_rate = sampled_batch
        inputs, labels = Variable(transformed_sequence.float().cuda()), Variable(growth_rate.float().cuda())

        optimizer.zero_grad()
        # forward + backward + optimize
        net.train()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistic

    error = 0
    total = 0
    
    net.eval()
    for batch in tqdm(val_data_loader):
      v_seq, X_v, y_v = batch
      v_pred = net(Variable(X_v.float().cuda()))
      total += y_v.size(0)
      raw_error = v_pred[:,0].data - y_v.float().cuda()
      error += (raw_error**2).sum()

      avg_mse = error / float(total)
      tqdm.write('avg_mse: %.3f' % avg_mse)

torch.save(net, 'saved_model.t7')

  0%|          | 0/13075 [00:00<?, ?it/s]Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1d3d318320>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/opt/conda/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Clien

RuntimeError: input and target shapes do not match: input [32 x 1], target [32] at /pytorch/aten/src/THCUNN/generic/MSECriterion.cu:12

# New Section

# New Section

Finally, after training has completed, we evaluate again on the validation set.

In [20]:
error = 0
total = 0
for batch in tqdm(val_data_loader):
                v_seq, X_v, y_v = batch
                v_pred = net(Variable(X_v.float().cuda()))
                total += y_v.size(0)
                raw_error = v_pred[:,0].data - y_v.float().cuda()
                error += (raw_error**2).sum()

avg_mse = error / float(total)

print("Validation error: {}".format(avg_mse))

100%|██████████| 1453/1453 [00:23<00:00, 62.48it/s]

Validation error: 0.4917738338773088


In [21]:
error = 0
total = 0
preds = []
for batch in tqdm(test_data_loader):
                v_seq, X_v, y_v = batch
                v_pred = net(Variable(X_v.float().cuda()))
                total += y_v.size(0)
                raw_error = v_pred[:,0].data - y_v.float().cuda()
                preds.extend(v_pred[:,0].data)
                error += (raw_error**2).sum()

avg_mse = error / float(total)

print("Test error: {}".format(avg_mse))

100%|██████████| 765/765 [00:12<00:00, 59.93it/s]

Test error: 0.43096563762283135


Add R^2 calculation here to make it comparable to Keras version.

In [22]:
actuals = [test_data[i][2] for i in range(len(test_data))]

In [17]:
!pip install scipy

In [23]:
import scipy.stats
r2 = scipy.stats.pearsonr(preds, actuals)[0]**2
print(r2)

0.6189750238366379


In [22]:
actuals_tr = [train_data[i][2] for i in range(len(train_data))]


In [ ]:
r2 = scipy.stats.pearsonr(preds, actuals)[0]**2
print(r2)